In [1]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q]
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
}

defined trait KnightsKnaves

# Puzzle 26 - Knight and Knave

In [8]:
/* Según este viejo problema, tres de los habitantes - A, B, C - se encontraban en un jardin.
Un extranjero pasó por allí y le preguntó a A, "¿Eres caballero o escudero?".
A respondió, pero tan confusamente, que el extranjero no pudo enterarse de lo que decía.
Entonces el extranjero preguntó a B, "¿Que ha dicho A?". Y B le respondió "A ha dicho que es escudero".
Pero en ese instante el tercer hombre, C, dijo "¡No creas a B, que está mintiendo!"

Pregunta: ¿que son B y C?

Opcion1: B - escudero
         C - escudero
SOLUCION CORRECTA ---------------->  Opcion2: B - escudero
                                              C - caballero
Opcion3: B - caballero
         C - escudero
Opcion4: B - caballero
         C - caballero  
*/         

In [8]:
/* Detalle de la Solucion:

Es imposible que un caballero o un escudero digan: "Yo soy escudero", porque un caballero emitiría el enunciado
falso de que él es escudero, y un escudero no emitiría el enunciado verdadero de que él es un escudero.
Por lo tanto, A nunca diría que era un escudero. Así, B mentía cuando dijo que A habia dicho que él era escudero.
Por lo tanto B es escudero. Puesto que C dijo que B estaba mintiendo, y B estaba ciertamente mintiendo, C dijo la verdad, 
de aquí que se tratara de un caballero. Asi pues, B es un escudero y C un caballero. (Es imposible saber que es A).

*/

In [8]:
/* Definicion en LPO:

A - 1
B - 2
C - 3

Says(P[2], Says(P[1], P[1] = Knave)) ∧ Says(P[3], P[2] = Knave)
B: A says he is a knave ---> Says(P[2], Says(P[1], P[1] = Knave))
C: B is a knave         ---> Says(P[3], P[2] = Knave)

Pseudo ejemplo:

Says(B, Says(A, knave(A))) ∧ Says(C, knave(B)) => (knave(B) ∧ knight(C))

*/

In [12]:
// VERSION 1 - SIGNATURA INCORRECTA 

/*
def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    (b.Says[a.Says[a.Knave]], c.Says[b.Knave]) => (b.Knave, c.Knight) = 
    ({t: (b.Says[a.Says[a.Knave]], c.Says[b.Knave]) =>
        
        ??? : (b.Knave, c.Knight)
        
    }): ((b.Says[a.Says[a.Knave]], c.Says[b.Knave]) => (b.Knave, c.Knight))
*/    
    

In [21]:
// VERSION 2
// Primero P1 con b en lugar de c - CON ESTE SI OK
def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight) = 
    ({t: ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) =>
        t._1: b.Says[a.Says[a.Knave]]
        t._2: c.Says[b.Knave]
        
        (premises.P1(b) match{
            case Left(bKnight: b.Knight) =>
                // def P2[P](b: Inhabitant): b.Knight => b.Says[P] => P     
                val b_L1: b.Knight => b.Says[a.Says[a.Knave]] => a.Says[a.Knave] = premises.P2[a.Says[a.Knave]](b)
                val b_L2: b.Says[a.Says[a.Knave]] => a.Says[a.Knave] = b_L1(bKnight)
                val b_L3: a.Says[a.Knave] = b_L2(t._1)
                
                (premises.P1(a) match{
                    case Left(aKnight: a.Knight) => 
                        // def P2[P](a: Inhabitant): a.Knight => a.Says[P] => P 
                        val a_L1: a.Knight => a.Says[a.Knave] => a.Knave = premises.P2[a.Knave](a)
                        val a_L2: a.Says[a.Knave] => a.Knave = a_L1(aKnight)
                        val a_L3: a.Knave = a_L2(b_L3)
                        a_L3(aKnight): Nothing
                        a_L3(aKnight) : (b.Knave, c.Knight)
                    case Right(aKnave: a.Knave) => 
                        //def P3[P](a: Inhabitant): a.Knave => a.Says[P] => ¬[P]
                        val a_R1: a.Knave => a.Says[a.Knave] => ¬[a.Knave] = premises.P3[a.Knave](a)
                        val a_R2: a.Says[a.Knave] => ¬[a.Knave] = a_R1(aKnave)
                        val a_R3: a.Knave => Nothing = a_R2(b_L3)
                        a_R3(aKnave): Nothing
                        a_R3(aKnave) : (b.Knave, c.Knight)
                }) : (b.Knave, c.Knight)
            
            case Right(bKnave: b.Knave) =>
                //def P3[P](b: Inhabitant): b.Knave => b.Says[P] => ¬[P]
                val b_R1: b.Knave => b.Says[a.Says[a.Knave]] => ¬[a.Says[a.Knave]] = premises.P3[a.Says[a.Knave]](b)
                val b_R2: b.Says[a.Says[a.Knave]] => ¬[a.Says[a.Knave]] = b_R1(bKnave)
                val b_R3: a.Says[a.Knave] => Nothing = b_R2(t._1)
            
                (premises.P1(c) match{
                    case Left(cKnight: c.Knight) => 
                        // def P2[P](c: Inhabitant): c.Knight => c.Says[P] => P 
                        val c_L1: c.Knight => c.Says[b.Knave] => b.Knave = premises.P2[b.Knave](c)
                        val c_L2: c.Says[b.Knave] => b.Knave = c_L1(cKnight)
                        val c_L3: b.Knave = c_L2(t._2)
                        (c_L2(t._2), cKnight) : (b.Knave, c.Knight)
                    case Right(cKnave: c.Knave) => 
                        //def P3[P](c: Inhabitant): c.Knave => c.Says[P] => ¬[P]
                        val c_R1: c.Knave => c.Says[b.Knave] => ¬[b.Knave] = premises.P3[b.Knave](c)
                        val c_R2: c.Says[b.Knave] => ¬[b.Knave] = c_R1(cKnave)
                        val c_R3: b.Knave => Nothing = c_R2(t._2)
                        
                        c_R3(bKnave): Nothing
                    
                        c_R3(bKnave) : (b.Knave, c.Knight)
                }) : (b.Knave, c.Knight)
        }) : (b.Knave, c.Knight)
        
    }): (((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight))
        
    
    

defined function puzzle26

In [22]:
// VERSION 3 - SIMPLIFICADO
// Primero P1 con b en lugar de c - CON ESTE SI OK
def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight) = 
    ({t =>
        (premises.P1(b) match{
            case Left(bKnight) =>
                // def P2[P](b: Inhabitant): b.Knight => b.Says[P] => P     
                val b_L1: b.Knight => b.Says[a.Says[a.Knave]] => a.Says[a.Knave] = premises.P2[a.Says[a.Knave]](b)
                val b_L2: b.Says[a.Says[a.Knave]] => a.Says[a.Knave] = b_L1(bKnight)
                val b_L3: a.Says[a.Knave] = b_L2(t._1)
                
                (premises.P1(a) match{
                    case Left(aKnight) => 
                        // def P2[P](a: Inhabitant): a.Knight => a.Says[P] => P 
                        val a_L1: a.Knight => a.Says[a.Knave] => a.Knave = premises.P2[a.Knave](a)
                        val a_L2: a.Says[a.Knave] => a.Knave = a_L1(aKnight)
                        val a_L3: a.Knave = a_L2(b_L3)
                        a_L3(aKnight)
                    case Right(aKnave) => 
                        //def P3[P](a: Inhabitant): a.Knave => a.Says[P] => ¬[P]
                        val a_R1: a.Knave => a.Says[a.Knave] => ¬[a.Knave] = premises.P3[a.Knave](a)
                        val a_R2: a.Says[a.Knave] => ¬[a.Knave] = a_R1(aKnave)
                        val a_R3: a.Knave => Nothing = a_R2(b_L3)
                        a_R3(aKnave)
                }) 
            
            case Right(bKnave) =>
                //def P3[P](b: Inhabitant): b.Knave => b.Says[P] => ¬[P]
                val b_R1: b.Knave => b.Says[a.Says[a.Knave]] => ¬[a.Says[a.Knave]] = premises.P3[a.Says[a.Knave]](b)
                val b_R2: b.Says[a.Says[a.Knave]] => ¬[a.Says[a.Knave]] = b_R1(bKnave)
                val b_R3: a.Says[a.Knave] => Nothing = b_R2(t._1)
            
                (premises.P1(c) match{
                    case Left(cKnight) => 
                        // def P2[P](c: Inhabitant): c.Knight => c.Says[P] => P 
                        val c_L1: c.Knight => c.Says[b.Knave] => b.Knave = premises.P2[b.Knave](c)
                        val c_L2: c.Says[b.Knave] => b.Knave = c_L1(cKnight)
                        val c_L3: b.Knave = c_L2(t._2)
                        (c_L2(t._2), cKnight)
                    case Right(cKnave) => 
                        //def P3[P](c: Inhabitant): c.Knave => c.Says[P] => ¬[P]
                        val c_R1: c.Knave => c.Says[b.Knave] => ¬[b.Knave] = premises.P3[b.Knave](c)
                        val c_R2: c.Says[b.Knave] => ¬[b.Knave] = c_R1(cKnave)
                        val c_R3: b.Knave => Nothing = c_R2(t._2)
                        c_R3(bKnave)
                }) 
        }) 
    })
        
    
    

defined function puzzle26

In [22]:
// Sugar version
// Primero P1 con b en lugar de c - CON ESTE SI OK

/*def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight) = 
    ({t =>
        (premises.P1(b) match{
            case Left(bKnight) =>
                // def P2[P](b: Inhabitant): b.Knight => b.Says[P] => P     
                val b_L1: b.Knight => b.Says[a.Says[a.Knave]] => a.Says[a.Knave] = premises.P2[a.Says[a.Knave]](b)
                val b_L2: b.Says[a.Says[a.Knave]] => a.Says[a.Knave] = b_L1(bKnight)
                val b_L3: a.Says[a.Knave] = b_L2(t._1)
                
                (premises.P1(a) match{
                    case Left(aKnight) => 
                        // def P2[P](a: Inhabitant): a.Knight => a.Says[P] => P 
                        val a_L1: a.Knight => a.Says[a.Knave] => a.Knave = premises.P2[a.Knave](a)
                        val a_L2: a.Says[a.Knave] => a.Knave = a_L1(aKnight)
                        val a_L3: a.Knave = a_L2(b_L3)
                        a_L3(aKnight)
                    case Right(aKnave) => 
                        //def P3[P](a: Inhabitant): a.Knave => a.Says[P] => ¬[P]
                        val a_R1: a.Knave => a.Says[a.Knave] => ¬[a.Knave] = premises.P3[a.Knave](a)
                        val a_R2: a.Says[a.Knave] => ¬[a.Knave] = a_R1(aKnave)
                        val a_R3: a.Knave => Nothing = a_R2(b_L3)
                        a_R3(aKnave)
                }) 
            
            case Right(bKnave) =>
                //def P3[P](b: Inhabitant): b.Knave => b.Says[P] => ¬[P]
                val b_R1: b.Knave => b.Says[a.Says[a.Knave]] => ¬[a.Says[a.Knave]] = premises.P3[a.Says[a.Knave]](b)
                val b_R2: b.Says[a.Says[a.Knave]] => ¬[a.Says[a.Knave]] = b_R1(bKnave)
                val b_R3: a.Says[a.Knave] => Nothing = b_R2(t._1)
            
                (premises.P1(c) match{
                    case Left(cKnight) => 
                        // def P2[P](c: Inhabitant): c.Knight => c.Says[P] => P 
                        val c_L1: c.Knight => c.Says[b.Knave] => b.Knave = premises.P2[b.Knave](c)
                        val c_L2: c.Says[b.Knave] => b.Knave = c_L1(cKnight)
                        val c_L3: b.Knave = c_L2(t._2)
                        (c_L2(t._2), cKnight)
                    case Right(cKnave) => 
                        //def P3[P](c: Inhabitant): c.Knave => c.Says[P] => ¬[P]
                        val c_R1: c.Knave => c.Says[b.Knave] => ¬[b.Knave] = premises.P3[b.Knave](c)
                        val c_R2: c.Says[b.Knave] => ¬[b.Knave] = c_R1(cKnave)
                        val c_R3: b.Knave => Nothing = c_R2(t._2)
                        c_R3(bKnave)
                }) 
        }) 
    })
*/

def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight) = 
        bSaycSay =>
            premises.eitherKnightOrKnave(b).fold(
                bIsKnight =>
                    val aux1 = premises.noKnightLies(b)(bIsKnight)(bSaycSay._1): a.Says[a.Knave]
                    premises.eitherKnightOrKnave(a).fold(
                        aIsKnight =>
                            val aux1_1 = premises.noKnightLies(a)(aIsKnight)(aux1): a.Knave
                            aux1_1(aIsKnight)
                        ,
                        aIsKnave => 
                            val aux1_2 = premises.noKnaveTellsTruth(a)(aIsKnave)(aux1): Not[a.Knave]
                            aux1_2(aIsKnave)
                    )
                ,
                bIsKnave =>
                    val aux2 = premises.noKnightLies(b)(bIsKnave)(bSaycSay._1): Not[a.Says[a.Knave]]
                    premises.eitherKnightOrKnave(c).fold(
                        cIsKnight =>
                            val aux2_1 = premises.noKnightLies(c)(cIsKnight)(bSaycSay._2): b.Knave
                            (aux2_1, cIsKnight)
                        ,
                        cIsKnave => 
                            val aux2_2 = premises.noKnaveTellsTruth(c)(cIsKnave)(bSaycSay._2): Not[b.Knave]
                            aux2_2(bIsKnave)
                    )
           )
    
    

(console):58:21 expected ")"
// Sugar version
// Primero P1 con b en lugar de c - CON ESTE SI OK

/*def puzzle26(premises: KnightsKnaves)(a: Inhabitant)(b: Inhabitant)(c: Inhabitant):
    ((b.Says[a.Says[a.Knave]], c.Says[b.Knave])) => (b.Knave, c.Knight) = 
    ({t =>
        (premises.P1(b) match{
            case Left(bKnight) =>
                // def P2[P](b: Inhabitant): b.Knight => b.Says[P] => P     
                val b_L1: b.Knight => b.Says[a.Says[a.Knave]] => a.Says[a.Knave] = premises.P2[a.Says[a.Knave]](b)
                val b_L2: b.Says[a.Says[a.Knave]] => a.Says[a.Knave] = b_L1(bKnight)
                val b_L3: a.Says[a.Knave] = b_L2(t._1)
                
                (premises.P1(a) match{
                    case Left(aKnight) => 
                        // def P2[P](a: Inhabitant): a.Knight => a.Says[P] => P 
                        val a_L1: a.Knight => a.Says[a.Knave] => a.Knave = premises.P2[a.Knave](a)
                        val a_L2: a.Says[a.Knave] => a

: 